In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import time
import csv
import json
import calendar
from datetime import date
import random

In [2]:
wardHHIncome = pd.read_csv("resources/Ward_HH_Income.csv").set_index("WardNumber")

In [3]:
def get_ward_data(latitude, longitude):
    try:
        wardnumber_2010 = 0
        ward_name_2010 = ""
        medianHHIncome = 0
        wardnumber_2018 = 0
        ward_name_2018 = ""
        objects = [{'name': '', 'centroid': {'coordinates': [0,0], 'type': 'Point'}}]
        
#         ward_2010_url = f"https://represent.opennorth.ca/boundaries/toronto-wards-2010/?contains={latitude},{longitude}"
#         ward_2018_url = f"https://represent.opennorth.ca/boundaries/toronto-wards-2018/?contains={latitude},{longitude}"
        ward_centroid_url = f"https://represent.opennorth.ca/boundaries/toronto-wards-2018/centroid?contains={latitude},{longitude}"
#         to_2010_wards = requests.get(ward_2010_url).json()
#         if len(to_2010_wards['objects']) > 0:
#             wardnumber_2010 = to_2010_wards['objects'][0]['external_id']
#             ward_name_2010 = to_2010_wards["objects"][0]["name"]
#             medianHHIncome = wardHHIncome.loc[int(wardnumber_2010)]["MedianHHIncome"]
        
        waitfor = random.randint(0,9)
        print(f"Sleeping for {waitfor} seconds")
#         time.sleep(waitfor)
        
#         to_2018_wards = requests.get(ward_2018_url).json()    
#         if len(to_2018_wards['objects']) > 0:
#             wardnumber_2018 = to_2018_wards['objects'][0]['external_id']
#             ward_name_2018 = to_2018_wards["objects"][0]["name"]
 
        to_ward_centroid = requests.get(ward_centroid_url).json() 
        if len(to_ward_centroid['objects'])>0:
            objects = to_ward_centroid['objects']
        
    except Exception as e:
        print(f"Failed to find boundaries for latitude {latitude} and longitude {longitude}\n{e}")
    
    ward_data = list(objects)
#     list([wardnumber_2010, ward_name_2010, medianHHIncome, wardnumber_2018, ward_name_2018])

    return ward_data

In [6]:
#Load full dataframe
mci_full_df = pd.read_csv("resources/MCI_2014_to_2018_full_new.csv").sort_values(["wardname 2018"])
# make sure Lat and Long are float types
mci_full_df["Lat"] = mci_full_df["Lat"].astype(float)
mci_full_df["Long"] = mci_full_df["Long"].astype(float)

In [7]:
def get_pivot(df, indexcol, pivotcol):
    # Group by both index and pivot columns and count the number of crimes

    df_groupby = df.groupby([pivotcol,indexcol])["ObjectId"].agg([("MCI count","count")]).\
                reset_index(None)
    
    # Set the index to indexcol                                          
    df_groupby = df_groupby.set_index(indexcol)
    
    # Pivot the grouped by dataframe on the pivotcol
    df_pivot = df_groupby.pivot(columns=pivotcol,values="MCI count")
    
    # Return both the dataframe and the pvalue for all the groups of values
    return df_pivot


In [16]:
df_pivot_ward = get_pivot(mci_full_df,"wardname 2018","MCI")
df_pivot_ward.to_csv("resources/ward_pivot_mci.csv")
df_pivot_ward

MCI,Assault,Auto Theft,Break and Enter,Robbery,Theft Over
wardname 2018,,,,,
Beaches-East York,3649,380,1438,716,168
Davenport,3358,470,1163,646,157
Don Valley East,2268,339,819,368,128
Don Valley North,2012,509,1048,337,144
Don Valley West,1424,363,1355,192,166
Eglinton-Lawrence,2648,1003,1746,650,265
Etobicoke Centre,1629,944,1241,419,136
Etobicoke North,4300,2523,1230,1269,304
Etobicoke-Lakeshore,3792,1049,1503,608,364


In [12]:
df_pivot_primise = get_pivot(mci_full_df,"premisetype","MCI")
df_pivot_primise.to_csv("resources/premise_pivot_mci.csv")
df_pivot_primise

MCI,Assault,Auto Theft,Break and Enter,Robbery,Theft Over
premisetype,,,,,
Apartment,28430,395,9207,1640,820
Commercial,14146,1795,10286,4139,1843
House,11628,4977,13141,526,965
Other,14109,476,2076,1468,411
Outside,22182,10421,11,10240,1337


In [14]:
df_pivot_tod = get_pivot(mci_full_df,"occurrencehour","MCI")
df_pivot_tod.to_csv("resources/tod_pivot_mci.csv")
df_pivot_tod

MCI,Assault,Auto Theft,Break and Enter,Robbery,Theft Over
occurrencehour,,,,,
0,5783,1181,2321,986,578
1,4125,551,1368,951,116
2,4493,420,1384,827,79
3,2834,307,1497,678,68
4,1717,258,1440,420,58
5,1227,283,1134,251,45
6,1204,329,893,226,59
7,1643,470,1060,157,101
8,2514,518,1658,200,151


In [15]:
df_pivot_dow= get_pivot(mci_full_df,"occurrencedayofweek","MCI")
df_pivot_dow.to_csv("resources/dow_pivot_mci.csv")
df_pivot_dow

MCI,Assault,Auto Theft,Break and Enter,Robbery,Theft Over
occurrencedayofweek,,,,,
Friday,12915,2787,5866,2816,921
Monday,12234,2525,5011,2431,769
Saturday,14489,2613,4525,2760,704
Sunday,14628,2373,4068,2457,630
Thursday,12129,2646,5237,2507,795
Tuesday,11773,2560,5020,2517,777
Wednesday,12286,2557,4993,2523,778


In [5]:
lat_long_df = mci_full_df[["Lat","Long"]].sort_values(["Lat", "Long"])

In [6]:
lat_long_df = lat_long_df.drop_duplicates()

In [7]:
lat_long_df

,Lat,Long
88461,43.587093,-79.539726
1403,43.587353,-79.541870
3410,43.587971,-79.541107
11005,43.588016,-79.538467
88509,43.588532,-79.539925
49634,43.588543,-79.537384
96190,43.588551,-79.533211
10587,43.588634,-79.542152
3614,43.588665,-79.543922
1039,43.588833,-79.536209


In [8]:
columns = ["Lat", "Long", "geometry"]
ward_lat_long_df = pd.DataFrame(columns=columns)
ward_lat_long_df

,Lat,Long,geometry


In [20]:
ward_lat_long_df["Lat"]=ward_lat_long_df["Lat"].astype(float)
ward_lat_long_df["Long"]=ward_lat_long_df["Long"].astype(float)

In [42]:
i = 0 
for coordinate in lat_long_df.itertuples():
    j=0
    while True:
        try:
            if len(ward_lat_long_df[(ward_lat_long_df["Lat"]==coordinate.Lat) & (ward_lat_long_df["Long"]==coordinate.Long)])==0:
                warddata = get_ward_data(coordinate.Lat, coordinate.Long)
    #             [0],warddata[1],warddata[2],warddata[3],warddata[4]
                lat_long_l = list([coordinate.Lat, coordinate.Long,warddata[0]])
                print(f"index: {i} | {lat_long_l}")
                ward_lat_long_df = pd.concat([ward_lat_long_df,pd.DataFrame([lat_long_l], columns=columns)])
            i = i + 1
            break
        except Exception as e:
            print(f"Re-try number: {j}\n{e}")
            print(warddata)
            j = j + 1
            if j > 100:
                break
            time.sleep(random.randint(0,9))


Sleeping for 7 seconds
index: 434 | [43.6164093, -79.3723755, {'name': '', 'centroid': {'coordinates': [0, 0], 'type': 'Point'}}]
Sleeping for 0 seconds
index: 2590 | [43.655838, -79.59841159999999, {'name': '', 'centroid': {'coordinates': [0, 0], 'type': 'Point'}}]
Sleeping for 4 seconds
index: 3309 | [43.6624489, -79.59114840000001, {'name': '', 'centroid': {'coordinates': [0, 0], 'type': 'Point'}}]
Sleeping for 3 seconds
index: 4730 | [43.6759682, -79.276619, {'name': 'Scarborough Southwest', 'centroid': {'coordinates': [-79.25978904250549, 43.7136280859651], 'type': 'Point'}}]
Sleeping for 4 seconds
index: 4731 | [43.6759872, -79.29684449999999, {'name': 'Beaches-East York', 'centroid': {'coordinates': [-79.30667407750994, 43.68927152233702], 'type': 'Point'}}]
Sleeping for 1 seconds
index: 4732 | [43.6760292, -79.5013504, {'name': 'York South-Weston', 'centroid': {'coordinates': [-79.49337260252571, 43.69495374466498], 'type': 'Point'}}]
Sleeping for 1 seconds
index: 4733 | [43.67

Sleeping for 5 seconds
index: 4773 | [43.6765213, -79.43222809999999, {'name': 'Davenport', 'centroid': {'coordinates': [-79.44405037130906, 43.66935483342086], 'type': 'Point'}}]
Sleeping for 3 seconds
index: 4774 | [43.6765289, -79.4743729, {'name': 'York South-Weston', 'centroid': {'coordinates': [-79.49337260252571, 43.69495374466498], 'type': 'Point'}}]
Sleeping for 3 seconds
index: 4775 | [43.6765289, -79.34910579999999, {'name': 'Toronto-Danforth', 'centroid': {'coordinates': [-79.33925166569755, 43.665980133695044], 'type': 'Point'}}]
Sleeping for 4 seconds
index: 4776 | [43.6765289, -79.3048859, {'name': 'Beaches-East York', 'centroid': {'coordinates': [-79.30667407750994, 43.68927152233702], 'type': 'Point'}}]
Sleeping for 4 seconds
index: 4777 | [43.676536600000006, -79.3814926, {'name': 'University-Rosedale', 'centroid': {'coordinates': [-79.39431224956856, 43.67114018210597], 'type': 'Point'}}]
Sleeping for 2 seconds
index: 4778 | [43.676536600000006, -79.32080079999999, {

index: 4818 | [43.676841700000004, -79.5642166, {'name': 'Etobicoke Centre', 'centroid': {'coordinates': [-79.55252701676034, 43.66443385100261], 'type': 'Point'}}]
Sleeping for 5 seconds
index: 4819 | [43.676841700000004, -79.5060349, {'name': 'York South-Weston', 'centroid': {'coordinates': [-79.49337260252571, 43.69495374466498], 'type': 'Point'}}]
Sleeping for 4 seconds
index: 4820 | [43.676841700000004, -79.44815059999999, {'name': 'Davenport', 'centroid': {'coordinates': [-79.44405037130906, 43.66935483342086], 'type': 'Point'}}]
Sleeping for 2 seconds
index: 4821 | [43.676841700000004, -79.4103622, {'name': "Toronto-St. Paul's", 'centroid': {'coordinates': [-79.4139873040851, 43.69053009074875], 'type': 'Point'}}]
Sleeping for 5 seconds
index: 4822 | [43.676841700000004, -79.4029846, {'name': "Toronto-St. Paul's", 'centroid': {'coordinates': [-79.4139873040851, 43.69053009074875], 'type': 'Point'}}]
Sleeping for 9 seconds
index: 4823 | [43.6768494, -79.44020079999999, {'name': '

Sleeping for 3 seconds
index: 4864 | [43.677227, -79.54522709999999, {'name': 'Etobicoke Centre', 'centroid': {'coordinates': [-79.55252701676034, 43.66443385100261], 'type': 'Point'}}]
Sleeping for 3 seconds
Failed to find boundaries for latitude 43.6772308 and longitude -79.47190859999999
Expecting value: line 1 column 1 (char 0)
index: 4865 | [43.6772308, -79.47190859999999, {'name': '', 'centroid': {'coordinates': [0, 0], 'type': 'Point'}}]
Sleeping for 3 seconds
index: 4866 | [43.677242299999996, -79.49614720000001, {'name': 'York South-Weston', 'centroid': {'coordinates': [-79.49337260252571, 43.69495374466498], 'type': 'Point'}}]
Sleeping for 8 seconds
index: 4867 | [43.6772461, -79.27960970000001, {'name': 'Scarborough Southwest', 'centroid': {'coordinates': [-79.25978904250549, 43.7136280859651], 'type': 'Point'}}]
Sleeping for 1 seconds
index: 4868 | [43.677269, -79.4462204, {'name': 'Davenport', 'centroid': {'coordinates': [-79.44405037130906, 43.66935483342086], 'type': 'Po

Failed to find boundaries for latitude 43.6775703 and longitude -79.4594345
Expecting value: line 1 column 1 (char 0)
index: 4907 | [43.6775703, -79.4594345, {'name': '', 'centroid': {'coordinates': [0, 0], 'type': 'Point'}}]
Sleeping for 2 seconds
index: 4908 | [43.677619899999996, -79.43746949999999, {'name': 'Davenport', 'centroid': {'coordinates': [-79.44405037130906, 43.66935483342086], 'type': 'Point'}}]
Sleeping for 2 seconds
index: 4909 | [43.6776276, -79.3751755, {'name': 'University-Rosedale', 'centroid': {'coordinates': [-79.39431224956856, 43.67114018210597], 'type': 'Point'}}]
Sleeping for 7 seconds
index: 4910 | [43.6776314, -79.5212631, {'name': 'Etobicoke Centre', 'centroid': {'coordinates': [-79.55252701676034, 43.66443385100261], 'type': 'Point'}}]
Sleeping for 6 seconds
index: 4911 | [43.677639, -79.3975906, {'name': 'University-Rosedale', 'centroid': {'coordinates': [-79.39431224956856, 43.67114018210597], 'type': 'Point'}}]
Sleeping for 9 seconds
index: 4912 | [43.

...

Sleeping for 9 seconds
index: 14645 | [43.7778587, -79.399826, {'name': 'Willowdale', 'centroid': {'coordinates': [-79.41649996767849, 43.77688740097211], 'type': 'Point'}}]
Sleeping for 2 seconds
index: 14648 | [43.77789310000001, -79.4150848, {'name': 'Willowdale', 'centroid': {'coordinates': [-79.41649996767849, 43.77688740097211], 'type': 'Point'}}]
Sleeping for 5 seconds
index: 14651 | [43.777946500000006, -79.4691238, {'name': 'York Centre', 'centroid': {'coordinates': [-79.46780765337418, 43.75256879776807], 'type': 'Point'}}]
Sleeping for 1 seconds
index: 14654 | [43.7779846, -79.311203, {'name': 'Scarborough-Agincourt', 'centroid': {'coordinates': [-79.3081363311284, 43.79648590354776], 'type': 'Point'}}]
Sleeping for 7 seconds
index: 14662 | [43.7781258, -79.4229813, {'name': 'Willowdale', 'centroid': {'coordinates': [-79.41649996767849, 43.77688740097211], 'type': 'Point'}}]
Sleeping for 9 seconds
index: 14694 | [43.778560600000006, -79.2222137, {'name': 'Scarborough-Guildwo

index: 15016 | [43.782802600000004, -79.2213745, {'name': 'Scarborough-Guildwood', 'centroid': {'coordinates': [-79.21466456778637, 43.765800495564754], 'type': 'Point'}}]
Sleeping for 0 seconds
index: 15018 | [43.7828598, -79.3600464, {'name': 'Don Valley North', 'centroid': {'coordinates': [-79.36003932715407, 43.78760308060638], 'type': 'Point'}}]
Sleeping for 5 seconds
index: 15021 | [43.7828751, -79.4104843, {'name': 'Willowdale', 'centroid': {'coordinates': [-79.41649996767849, 43.77688740097211], 'type': 'Point'}}]
Sleeping for 7 seconds
index: 15023 | [43.7828865, -79.4365463, {'name': 'Willowdale', 'centroid': {'coordinates': [-79.41649996767849, 43.77688740097211], 'type': 'Point'}}]
Sleeping for 4 seconds
index: 15026 | [43.7829895, -79.3362198, {'name': 'Don Valley North', 'centroid': {'coordinates': [-79.36003932715407, 43.78760308060638], 'type': 'Point'}}]
Sleeping for 7 seconds
index: 15028 | [43.783027600000004, -79.4589691, {'name': 'York Centre', 'centroid': {'coordi

index: 15122 | [43.78431320000001, -79.3857956, {'name': 'Don Valley North', 'centroid': {'coordinates': [-79.36003932715407, 43.78760308060638], 'type': 'Point'}}]
Sleeping for 4 seconds
index: 15125 | [43.7843285, -79.1602631, {'name': 'Scarborough-Rouge Park', 'centroid': {'coordinates': [-79.17685103921572, 43.80565418102799], 'type': 'Point'}}]
Sleeping for 2 seconds
index: 15128 | [43.78443529999999, -79.321373, {'name': 'Scarborough-Agincourt', 'centroid': {'coordinates': [-79.3081363311284, 43.79648590354776], 'type': 'Point'}}]
Sleeping for 8 seconds
index: 15130 | [43.7844658, -79.13320159999999, {'name': 'Scarborough-Rouge Park', 'centroid': {'coordinates': [-79.17685103921572, 43.80565418102799], 'type': 'Point'}}]
Sleeping for 9 seconds
index: 15132 | [43.7845116, -79.1839676, {'name': 'Scarborough-Rouge Park', 'centroid': {'coordinates': [-79.17685103921572, 43.80565418102799], 'type': 'Point'}}]
Sleeping for 0 seconds
index: 15135 | [43.784530600000004, -79.3258972, {'na

Sleeping for 7 seconds
index: 16649 | [43.8077126, -79.37609859999999, {'name': '', 'centroid': {'coordinates': [0, 0], 'type': 'Point'}}]
Sleeping for 7 seconds
index: 16683 | [43.8085098, -79.3725052, {'name': '', 'centroid': {'coordinates': [0, 0], 'type': 'Point'}}]
Sleeping for 4 seconds
index: 16710 | [43.8089676, -79.37043759999999, {'name': '', 'centroid': {'coordinates': [0, 0], 'type': 'Point'}}]
Sleeping for 8 seconds
index: 16782 | [43.810722399999996, -79.3627853, {'name': '', 'centroid': {'coordinates': [0, 0], 'type': 'Point'}}]
Sleeping for 5 seconds
index: 16803 | [43.811187700000005, -79.36077879999999, {'name': '', 'centroid': {'coordinates': [0, 0], 'type': 'Point'}}]
Sleeping for 7 seconds
index: 16837 | [43.8119202, -79.3575516, {'name': '', 'centroid': {'coordinates': [0, 0], 'type': 'Point'}}]
Sleeping for 7 seconds
index: 16861 | [43.81233220000001, -79.32157140000001, {'name': 'Scarborough-Agincourt', 'centroid': {'coordinates': [-79.3081363311284, 43.79648590

Sleeping for 3 seconds
index: 17208 | [43.8249817, -79.30152890000001, {'name': '', 'centroid': {'coordinates': [0, 0], 'type': 'Point'}}]
Sleeping for 1 seconds
index: 17222 | [43.825321200000005, -79.3000717, {'name': '', 'centroid': {'coordinates': [0, 0], 'type': 'Point'}}]
Sleeping for 6 seconds
index: 17232 | [43.825943, -79.2832031, {'name': 'Scarborough North', 'centroid': {'coordinates': [-79.25467034276721, 43.80967391853659], 'type': 'Point'}}]
Sleeping for 3 seconds
index: 17233 | [43.8259544, -79.29733279999999, {'name': '', 'centroid': {'coordinates': [0, 0], 'type': 'Point'}}]
Sleeping for 8 seconds
index: 17235 | [43.826080299999994, -79.2880783, {'name': 'Scarborough North', 'centroid': {'coordinates': [-79.25467034276721, 43.80967391853659], 'type': 'Point'}}]
Sleeping for 8 seconds
index: 17237 | [43.8260956, -79.2781372, {'name': 'Scarborough North', 'centroid': {'coordinates': [-79.25467034276721, 43.80967391853659], 'type': 'Point'}}]
Sleeping for 6 seconds
index:

In [36]:
ward_lat_long_df
# [ward_lat_long_df["geometry"]=="{'name': '', 'centroid': {'coordinates': [0,0], 'type': 'Point'}}"]

,Lat,Long,geometry
0,43.587093,-79.539726,"{'name': 'Etobicoke-Lakeshore', 'centroid': {'..."
0,43.587353,-79.541870,"{'name': 'Etobicoke-Lakeshore', 'centroid': {'..."
0,43.587971,-79.541107,"{'name': 'Etobicoke-Lakeshore', 'centroid': {'..."
0,43.588016,-79.538467,"{'name': 'Etobicoke-Lakeshore', 'centroid': {'..."
0,43.588532,-79.539925,"{'name': 'Etobicoke-Lakeshore', 'centroid': {'..."
0,43.588543,-79.537384,"{'name': 'Etobicoke-Lakeshore', 'centroid': {'..."
0,43.588551,-79.533211,"{'name': 'Etobicoke-Lakeshore', 'centroid': {'..."
0,43.588634,-79.542152,"{'name': 'Etobicoke-Lakeshore', 'centroid': {'..."
0,43.588665,-79.543922,"{'name': 'Etobicoke-Lakeshore', 'centroid': {'..."
0,43.588833,-79.536209,"{'name': 'Etobicoke-Lakeshore', 'centroid': {'..."


In [85]:
wardHHIncome2018["wardnumber 2018"] = wardHHIncome2018["wardnumber 2018"].astype(int)

KeyError: 'wardnumber 2018'

In [ ]:
ward_lat_long_df.count()
ward_lat_long_df["wardmedian 2010"] = ward_lat_long_df["wardmedian 2010"].astype(float)
# ward_lat_long_df[ward_lat_long_df["wardnumber 2018"]>0]


In [ ]:
lat_long_df =  ward_lat_long_df[ward_lat_long_df["wardnumber 2018"]==0][["Lat","Long"]].\
                    drop_duplicates().\
                    sort_values(["Lat", "Long"])

In [ ]:
lat_long_df.count()


In [ ]:
# ward_lat_long_df = pd.read_csv("resources/ward_lat_long.csv")

In [43]:
ward_lat_long_df = pd.read_csv("resources/ward_lat_long_geo_point.csv")

In [39]:
ward_lat_long_df.count()

Lat         14032
Long        14032
geometry    14032
dtype: int64

In [40]:
lat_long_df.count()

Lat     17330
Long    17330
dtype: int64

In [41]:
i = 0 
for coordinate in lat_long_df.itertuples():
    j=0
    while True:
        try:
            warddata = get_ward_data(coordinate.Lat, coordinate.Long)
            lat_long_l = list([coordinate.Lat, coordinate.Long,warddata[0],warddata[1],warddata[2],warddata[3],warddata[4]])
            print(f"index: {i} | {lat_long_l}")
            ward_lat_long_df = pd.concat([ward_lat_long_df,pd.DataFrame([lat_long_l], columns=columns)])
            i = i + 1
            break
        except Exception as e:
            print(f"Re-try number: {j}\n{e}")
            print(warddata)
            j = j + 1
            if j > 100:
                break
            time.sleep(random.randint(0,9))


Sleeping for 6 seconds
Re-try number: 0
list index out of range
[{'name': 'Etobicoke-Lakeshore', 'centroid': {'coordinates': [-79.52087054631912, 43.62164878864557], 'type': 'Point'}}]
Sleeping for 8 seconds
Re-try number: 1
list index out of range
[{'name': 'Etobicoke-Lakeshore', 'centroid': {'coordinates': [-79.52087054631912, 43.62164878864557], 'type': 'Point'}}]
Sleeping for 7 seconds
Re-try number: 2
list index out of range
[{'name': 'Etobicoke-Lakeshore', 'centroid': {'coordinates': [-79.52087054631912, 43.62164878864557], 'type': 'Point'}}]
Sleeping for 5 seconds
Re-try number: 3
list index out of range
[{'name': 'Etobicoke-Lakeshore', 'centroid': {'coordinates': [-79.52087054631912, 43.62164878864557], 'type': 'Point'}}]
Sleeping for 7 seconds
Re-try number: 4
list index out of range
[{'name': 'Etobicoke-Lakeshore', 'centroid': {'coordinates': [-79.52087054631912, 43.62164878864557], 'type': 'Point'}}]
Sleeping for 3 seconds
Failed to find boundaries for latitude 43.5870934 an

KeyboardInterrupt: 

In [ ]:
wardHHIncome2018 = ward_lat_long_df[["wardmedian 2010", "wardnumber 2018"]].\
                sort_values(["wardnumber 2018", "wardmedian 2010"]).\
                drop_duplicates().\
                groupby(["wardnumber 2018"])["wardmedian 2010"].\
                agg(([("wardmedian 2018","median")]))
# wardHHIncome2018 = wardHHIncome2018"
wardHHIncome2018.count()
wardHHIncome2018.reset_index().drop_duplicates()
# wardHHIncome_2018 = ward_lat_long_df.groupby(["wardnumber 2018"])["wardmedian 2010"].\
#                 agg(([("wardmedian 2018","median")]))
# wardHHIncome2018 = wardHHIncome2018"
# wardHHIncome2018.count()
# wardHHIncome_2018.reset_index().drop_duplicates()
# wardHHIncome_2018["wardnumber 2018"]=wardHHIncome_2018["wardnumber 2018"].astype(int)
wardHHIncome_2018.reset_index()

In [ ]:
wardHHIncome_2018.dtypes

In [ ]:
ward_lat_long_df[ward_lat_long_df["wardnumber 2018"]==0][["Lat","Long"]].\
                    drop_duplicates().\
                    sort_values(["Lat", "Long"])

In [ ]:
ward_lat_long_df = ward_lat_long_df.sort_values(["Lat","Long"])

In [ ]:
#lat_long_df = ward_lat_long_df.loc[ward_lat_long_df["wardmedian"]==0][["Lat","Long"]]

In [ ]:
#lat_long_df = lat_long_df.drop_duplicates()

In [ ]:
#lat_long_df.head(10)

In [ ]:
#ward_lat_long_df = ward_lat_long_df.sort_values(["wardmedian"])

In [ ]:
#ward_lat_long_df

In [ ]:
#ward_lat_long_df = ward_lat_long_df.loc[ward_lat_long_df['wardmedian']!=0]

In [ ]:
#ward_lat_long_df = ward_lat_long_df.drop_duplicates()

In [ ]:
#ward_lat_long_df = ward_lat_long_df.loc[ward_lat_long_df['wardmedian']!=0]

In [37]:
# ward_lat_long_df.to_csv("resources/ward_lat_long_geo_point.csv")

In [ ]:
#ward_lat_long_df = ward_lat_long_df.drop_duplicates()

In [ ]:
#ward_lat_long_df.loc[ward_lat_long_df['wardmedian']!=0].count()

In [ ]:
ward_lat_long_df1 = pd.merge(ward_lat_long_df,wardHHIncome2018,how='inner',on=["wardnumber 2018"])

In [ ]:
ward_lat_long_df1

In [ ]:
mci_full_df1 = pd.merge(mci_full_df,ward_lat_long_df1,how='inner',on=["Lat","Long"])

In [ ]:
mci_full_df1 = mci_full_df1.drop_duplicates()

In [ ]:
mci_full_df1.to_csv("resources/MCI_2014_to_2018_full_new.csv")

In [ ]:
mci_full_df.count()

In [ ]:
mci_full_df1.head()

In [ ]:
mci_full_df1 = pd.read_csv("resources/MCI_2014_to_2018_full_new.csv")

In [ ]:
mci_full_df1["wardmedian 2018"] = mci_full_df1["wardmedian 2018"].astype(float)
mci_full_df1["wardnumber 2018"] = mci_full_df1["wardnumber 2018"].astype(int)


In [ ]:
mci_wardHHIncome2018 = mci_full_df1.groupby(["wardnumber 2018", "MCI"])["ObjectId"].\
                agg(([("MCI count","count")]))
# wardHHIncome2018 = wardHHIncome2018"
# mci_wardHHIncome2018["wardnumber 2018"] = mci_wardHHIncome2018["wardnumber 2018"].astype(int)
mci_wardHHIncome2018.reset_index()

In [ ]:
mci_wardHHIncome2018 = pd.merge(mci_wardHHIncome2018,wardHHIncome_2018,how='inner',on=["wardnumber 2018"])

In [ ]:
mci_wardHHIncome2018.dtypes



In [ ]:
wardHHIncome_2018.dtypes

In [ ]:
wardHHIncome_2018.to_csv("resources/wardHHIncome_2018.csv")

In [ ]:
mci_wardHHIncome2018.to_csv("resources/mci_wardHHIncome2018.csv")

In [ ]:
wardHHIncomce2018_df = pd.read_csv("resources/wardHHIncome_2018.csv")

In [ ]:
wardHHIncomce2018_df.dtypes

In [ ]:
mci_wardHHIncome2018_df = pd.read_csv("resources/mci_wardHHIncome2018.csv")

In [ ]:
mci_wardHHIncome2018_df.dtypes

In [ ]:
mciwardHHIncome2018_df = pd.merge(mci_wardHHIncome2018_df,wardHHIncomce2018_df,how='inner',on=["wardnumber 2018"])

In [ ]:
mciwardHHIncome2018_df.to_csv("resources/mciwardHHIncome2018.csv")

In [3]:
ward_mci_total_df = pd.read_csv("resources/ward_mci_total.csv")

In [5]:
ward_mci_total_df.quantile([])

,mci_total
count,25.000000
mean,6659.560000
std,2683.268934
min,3500.000000
25%,4193.000000
50%,6351.000000
75%,7884.000000
max,14346.000000


In [7]:
ward_mci_total_df["percentile"] = ward_mci_total_df["mci_total"].rank(pct=True)

In [8]:
ward_mci_total_df

,ward,mci_total,percentile
0,ward 1,9626,0.88
1,ward 2,4369,0.32
2,ward 3,7316,0.64
3,ward 4,5524,0.40
4,ward 5,7384,0.68
5,ward 6,6392,0.56
6,ward 7,9288,0.84
7,ward 8,6312,0.48
8,ward 9,5794,0.44
9,ward 10,10259,0.92


In [9]:
ward_mci_total_df.to_csv("resources/ward_mci_total.csv")